In [44]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
#Importing dataset with selected FIFA 2018 soccer players
file = "Raw Data Files/wc2018-players.csv"
WC_Players_DF = pd.read_csv(file)
WC_Players_DF.head()

,Team,#,Pos.,FIFA Popular Name,Birth Date,Shirt Name,Club,Height,Weight
0,Argentina,3,DF,TAGLIAFICO Nicolas,31.08.1992,TAGLIAFICO,AFC Ajax (NED),169,65
1,Argentina,22,MF,PAVON Cristian,21.01.1996,PAVÓN,CA Boca Juniors (ARG),169,65
2,Argentina,15,MF,LANZINI Manuel,15.02.1993,LANZINI,West Ham United FC (ENG),167,66
3,Argentina,18,DF,SALVIO Eduardo,13.07.1990,SALVIO,SL Benfica (POR),167,69
4,Argentina,10,FW,MESSI Lionel,24.06.1987,MESSI,FC Barcelona (ESP),170,72


In [45]:
WC_Players_DF["FIFA Popular Name"]=WC_Players_DF["FIFA Popular Name"].str.title()
WC_Players_DF["Shirt Name"]=WC_Players_DF["Shirt Name"].str.title()

In [46]:
WC_Players_DF['Birth Date']=pd.to_datetime(WC_Players_DF['Birth Date'], format="%d.%m.%Y")
WC_Players_DF.style.format({"Birth Date": lambda t: t.strftime("%m/%d/%Y")})
WC_Players_DF["Team"].replace(to_replace=["IR Iran"],value="Iran",inplace=True)
WC_Players_DF.head()

,Team,#,Pos.,FIFA Popular Name,Birth Date,Shirt Name,Club,Height,Weight
0,Argentina,3,DF,Tagliafico Nicolas,1992-08-31,Tagliafico,AFC Ajax (NED),169,65
1,Argentina,22,MF,Pavon Cristian,1996-01-21,Pavón,CA Boca Juniors (ARG),169,65
2,Argentina,15,MF,Lanzini Manuel,1993-02-15,Lanzini,West Ham United FC (ENG),167,66
3,Argentina,18,DF,Salvio Eduardo,1990-07-13,Salvio,SL Benfica (POR),167,69
4,Argentina,10,FW,Messi Lionel,1987-06-24,Messi,FC Barcelona (ESP),170,72


In [47]:
#Importing second dataset with all soccer players in a nation
CompletePlayerDF=pd.read_csv("Raw Data Files/completecCLEAN_FEW_COLUMNS.csv")
#Importing third dataset with Results of FIFA games
Results_DF = pd.read_csv("Raw Data Files/results.csv")

In [48]:
#Cleaning second dataset
CompletePlayerDF['birth_date']=pd.to_datetime(CompletePlayerDF['birth_date'], format="%m/%d/%Y")
CompletePlayerDF.style.format({"birth_date": lambda t: t.strftime("%m-%d-%Y")})
CompletePlayerDF.head()

,ID,name,age,nationality,overall,potential,club,birth_date,eur_value,eur_wage
0,20801,Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,1985-02-05,95500000,565000
1,158023,L. Messi,30,Argentina,93,93,FC Barcelona,1987-06-24,105000000,565000
2,190871,Neymar,25,Brazil,92,94,Paris Saint-Germain,1992-02-05,123000000,280000
3,176580,L. Suárez,30,Uruguay,92,92,FC Barcelona,1987-01-24,97000000,510000
4,167495,M. Neuer,31,Germany,92,92,FC Bayern Munich,1986-03-27,61000000,230000


In [49]:
#Clean third dataset
Results_DF['date']=pd.to_datetime(Results_DF['date'], format="%Y-%m-%d")
Results_DF['year'] = pd.DatetimeIndex(Results_DF['date']).year
Results_DF
#to have additional column with 2018 as year 
Results_Clean=Results_DF.loc[(Results_DF['tournament']=='FIFA World Cup') & (Results_DF['year'] ==2018)]
Results_Clean.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
39885,2018-06-14,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia,False,2018
39886,2018-06-15,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia,True,2018
39887,2018-06-15,Morocco,Iran,0,1,FIFA World Cup,St. Petersburg,Russia,True,2018
39888,2018-06-15,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia,True,2018
39889,2018-06-16,France,Australia,2,1,FIFA World Cup,Kazan,Russia,True,2018


In [50]:
#Calculate the winning teams in FIFA2018
pd.options.mode.chained_assignment = None 
Results_Clean['winning_team'] = Results_Clean['away_team'].where(Results_Clean['away_score'] >= Results_Clean['home_score'], Results_Clean['home_team'])
Results_Clean.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,winning_team
39885,2018-06-14,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia,False,2018,Russia
39886,2018-06-15,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia,True,2018,Uruguay
39887,2018-06-15,Morocco,Iran,0,1,FIFA World Cup,St. Petersburg,Russia,True,2018,Iran
39888,2018-06-15,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia,True,2018,Spain
39889,2018-06-16,France,Australia,2,1,FIFA World Cup,Kazan,Russia,True,2018,France


In [51]:
#Calculating each countries wins in the tournament
Results_group = Results_Clean.groupby('winning_team')['winning_team'].count()
total_wins = pd.DataFrame(Results_group)

total_wins.rename(columns = {'winning_team':'wins'}, inplace = True)
total_wins.head()

,wins
winning_team,
Argentina,1
Australia,1
Belgium,6
Brazil,3
Colombia,2


In [52]:
#Joining first two datasets to obtain players who played in fifa2018 and their overall score
players_score_df = pd.merge(WC_Players_DF, CompletePlayerDF , how='left', left_on=['Birth Date','Team'], right_on = ['birth_date','nationality'])
players_score_df.head()

#cleaning the output by removing unwanted fields
Joined_df=players_score_df.drop(columns=['Pos.', 'Club','club','nationality','birth_date','name'])
Joined_df.head()

,Team,#,FIFA Popular Name,Birth Date,Shirt Name,Height,Weight,ID,age,overall,potential,eur_value,eur_wage
0,Argentina,3,Tagliafico Nicolas,1992-08-31,Tagliafico,169,65,211256.0,24.0,79.0,84.0,14000000.0,23000.0
1,Argentina,22,Pavon Cristian,1996-01-21,Pavón,169,65,224254.0,21.0,75.0,84.0,10500000.0,20000.0
2,Argentina,15,Lanzini Manuel,1993-02-15,Lanzini,167,66,188988.0,24.0,81.0,87.0,24500000.0,95000.0
3,Argentina,18,Salvio Eduardo,1990-07-13,Salvio,167,69,190972.0,26.0,80.0,80.0,16000000.0,17000.0
4,Argentina,10,Messi Lionel,1987-06-24,Messi,170,72,158023.0,30.0,93.0,93.0,105000000.0,565000.0


In [53]:
#Calculating the team and their players score respectively 
group_df = Joined_df.groupby(['Team'])
group_team_df=group_df["overall"].sum().reset_index()
group_team_df.head()

,Team,overall
0,Argentina,2081.0
1,Australia,1571.0
2,Belgium,1811.0
3,Brazil,1881.0
4,Colombia,1747.0


In [54]:
#Final Dataframe with all the details needed for our analysis
Finals_df = pd.merge(Joined_df, Results_Clean , how='inner', left_on=['Team'], right_on = ['winning_team'])
Finals_df.head()

,Team,#,FIFA Popular Name,Birth Date,Shirt Name,Height,Weight,ID,age,overall,...,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,winning_team
0,Argentina,3,Tagliafico Nicolas,1992-08-31,Tagliafico,169,65,211256.0,24.0,79.0,...,Nigeria,Argentina,1,2,FIFA World Cup,St. Petersburg,Russia,True,2018,Argentina
1,Argentina,22,Pavon Cristian,1996-01-21,Pavón,169,65,224254.0,21.0,75.0,...,Nigeria,Argentina,1,2,FIFA World Cup,St. Petersburg,Russia,True,2018,Argentina
2,Argentina,15,Lanzini Manuel,1993-02-15,Lanzini,167,66,188988.0,24.0,81.0,...,Nigeria,Argentina,1,2,FIFA World Cup,St. Petersburg,Russia,True,2018,Argentina
3,Argentina,18,Salvio Eduardo,1990-07-13,Salvio,167,69,190972.0,26.0,80.0,...,Nigeria,Argentina,1,2,FIFA World Cup,St. Petersburg,Russia,True,2018,Argentina
4,Argentina,10,Messi Lionel,1987-06-24,Messi,170,72,158023.0,30.0,93.0,...,Nigeria,Argentina,1,2,FIFA World Cup,St. Petersburg,Russia,True,2018,Argentina


In [55]:
#Analysis
output_df = pd.merge(total_wins, group_team_df , how='left', left_on=['winning_team'], right_on = ['Team'])
output_df.head()

,wins,Team,overall
0,1,Argentina,2081.0
1,1,Australia,1571.0
2,6,Belgium,1811.0
3,3,Brazil,1881.0
4,2,Colombia,1747.0


In [56]:
output_df.rename(columns = {'Team':'team'}, inplace = True)

In [57]:

connection_string = f"postgres:{password}@localhost:5432/ETL"
engine = create_engine(f'postgresql://{connection_string}')

In [58]:
engine.execute("CREATE TABLE fifa2018(wins int, team varchar(60), overall float)")

In [59]:
output_df.to_sql(name='fifa2018', con=engine, if_exists='append', index=False)

In [60]:
teams = engine.execute("SELECT fifa2018.wins, fifa2018.team, fifa2018.overall FROM fifa2018 LIMIT 10")
for t in teams:
    print(t)

(1, 'Argentina', 2081.0)
(1, 'Australia', 1571.0)
(6, 'Belgium', 1811.0)
(3, 'Brazil', 1881.0)
(2, 'Colombia', 1747.0)
(1, 'Costa Rica', 1387.0)
(5, 'Croatia', 1521.0)
(2, 'Denmark', 1943.0)
(4, 'England', 2227.0)
(7, 'France', 2097.0)
